# Create Consumption Pricing objects in ADLSTE_COA

In [53]:
%connect name=Transcend

Connected: 'Transcend' connection activated for user 'sh186014'


<br/><br/><br/>
## COA Populated Tables - Consumption Pricing

In [82]:
drop table adlste_coa_stg.stg_dat_consumption_vantage_units

Success: 26 rows affected

In [83]:
-- GTT Table
CREATE GLOBAL TEMPORARY TABLE adlste_coa_stg.stg_dat_consumption_vantage_units 
( Site_ID  VARCHAR(128) CHARACTER SET UNICODE 
, LogDate  DATE         FORMAT 'YYYY-MM-DD'
, LogHour  INTEGER
, CPU_VU   DECIMAL(18,2)
, IO_VU    DECIMAL(18,2)
) Primary Index (Site_ID, LogDate)
ON COMMIT PRESERVE ROWS;

Success: 0 rows affected

In [75]:
rename table adlste_coa.coat_dat_consumption_vantage_units to adlste_coa.coat_dat_consumption_vantage_units_bkup

Success: 2 rows affected

In [76]:
-- TARGET Table
CREATE TABLE adlste_coa.coat_dat_consumption_vantage_units 
( Site_ID  VARCHAR(128) CHARACTER SET UNICODE 
, LogDate  DATE         FORMAT 'YYYY-MM-DD'
, LogHour  INTEGER
, CPU_VU   DECIMAL(18,2)
, IO_VU    DECIMAL(18,2)
, Run_Id   INTEGER
) Primary Index (Site_ID, LogDate);

Success: 0 rows affected

In [77]:
insert into adlste_coa.coat_dat_consumption_vantage_units
select * from adlste_coa.coat_dat_consumption_vantage_units_bkup

Success: 28203 rows affected

In [78]:
drop table adlste_coa.coat_dat_consumption_vantage_units_bkup

Success: 28 rows affected

In [84]:
replace view adlste_coa.coa_dat_consumption_vantage_units
as locking row for access
select * from adlste_coa.coat_dat_consumption_vantage_units;

Success: 7 rows affected

In [80]:
REPLACE PROCEDURE adlste_coa.sp_dat_consumption_vantage_units ( spversion VARCHAR(128) ) 
SQL SECURITY CREATOR
BEGIN 

    /* start setup audit process */
    
    DECLARE runid  INTEGER;
    DECLARE siteid VARCHAR(128);
    DECLARE tablename VARCHAR(128);
    DECLARE callingsp VARCHAR(128); 
    DECLARE startdate DATE;
    DECLARE enddate DATE;
    
    SET runid = 0;
    SET tablename = 'adlste_coa.coa_dat_consumption_vantage_units';
    SET callingsp = 'adlste_coa.sp_dat_consumption_vantage_units';
    SET siteid = 'Empty Table';
    SET startdate = DATE;
    SET enddate = DATE;

    SELECT 
     coalesce(Site_ID,'empty table')       
    ,min(LogDate) as StartDate
    ,max(LogDate) as EndDate
    INTO siteid, startdate, enddate
    FROM adlste_coa_stg.stg_dat_consumption_vantage_units
    Group by 1
    ;
    
    /* end setup audit process */
    
    
    
    CALL adlste_coa.sp_audit_log(runid, :siteid, :tablename, :callingsp, :spversion, :startdate, :enddate,
                              'normal run', 'START',  '') ;

    DELETE FROM adlste_coa.coat_dat_consumption_vantage_units WHERE (Site_ID, LogDate ) IN
    (Select Site_ID, LogDate From adlste_coa_stg.stg_dat_consumption_vantage_units );

    Insert into adlste_coa.coat_dat_consumption_vantage_units 
    SELECT
      Site_ID
    , LogDate
    , LogHour
    , CPU_VU
    , IO_VU
    , :runid
    FROM adlste_coa_stg.stg_dat_consumption_vantage_units;

    delete from adlste_coa_stg.stg_dat_consumption_vantage_units;
        
    CALL adlste_coa.sp_audit_log(runid, :siteid, :tablename, :callingsp, :spversion, :startdate, :enddate,
                              'normal run', 'END', '') ;

END;

Success: 0 rows affected

In [85]:
call adlste_coa.sp_dat_consumption_vantage_units('purge')

Success: 0 rows affected

# Test:

In [30]:
delete from adlste_coa_stg.stg_dat_consumption_vantage_units ;

insert into adlste_coa_stg.stg_dat_consumption_vantage_units 
select
 '{siteid}' as Site_ID,
 cast(coalesce(vu1.logdate, vu4.logdate) as format 'Y4-MM-DD') as LogDate,
 coalesce(vu1.loghr, vu4.loghr) as LogHour,
 zeroifnull(vu1.vantageunithrs) + zeroifnull(vu4.vantageunitutilityhrs) (format 'ZZ9.999') as CPU_VU,
 zeroifnull(vu1.vantageunitTB) + zeroifnull(vu4.vantageunitutilityTB) (format 'ZZ9.999') as IO_VU
from
    (select logdate,
     extract(hour from firstresptime) as LogHr,
     zeroifnull(sum(ampcputime + parsercputime + discputime)) / 3600 (format 'ZZ9.999') as VantageUnitHrs,
     zeroifnull(sum(reqiokb)) / (1024*1024*1024) (format 'ZZ9.999') as VantageUnitTB
    from
    pdcrinfo.dbqlogtbl_hst
    where logdate between DATE-91 and DATE-1
      and errorcode not in (2631, 2646, 3610, 3702, 3710, 3711, 5405, 7453, 7487, 7583, 7596, 9124, 9990)
      and username not in ('vcmuser','PDCRAccess','LockLogShredder','PDCRTPCD','console','tdap_admin','TDPUSER','tdwm',
                           'PDCRAdmin','SystemFe','PDCRCanary1M','PDCRCanary3M','td_ffe_svc_acct','PDCRCanary4M','PDCRCanary0M',
                           'PDCR users','PDCRCanary2M','TDMaps','SysAdmin','Crashdumps','Sys_Calendar')
    group by 1, 2) vu1
full outer join
    (select LogDate, LogHr,
     sum(Phase0IO) as TotalIOP0,
     sum(Phase1IO) as TotalIOP1,
     sum(Phase2IO) as TotalIOP2,
     sum(Phase3IO) as TotalIOP3,
     sum(Phase4IO) as TotalIOP4,
     sum(Phase0CPU) as TotalCPUP0,
     sum(Phase1CPU) as TotalCPUP1,
     sum(Phase2CPU) as TotalCPUP2,
     sum(Phase3CPU) as TotalCPUP3,
     sum(Phase4CPU) as TotalCPUP4,
     zeroifnull(TotalIOP0 + TotalIOP1  + TotalIOP2 + TotalIOP3 + TotalIOP4) / (1024*1024*1024) (format 'ZZ9.999') as VantageUnitUtilityTB,
     zeroifnull(TotalCPUP0 + TotalCPUP1  + TotalCPUP2 + TotalCPUP3 + TotalCPUP4)/3600 (format 'ZZ9.999') as VantageUnitUtilityHrs
    from
    (select
      cast(jobendtime as date) as LogDate,
      extract(hour from jobendtime) as LogHr,
      0 as Phase0CPU,
      0 as Phase0IO,
      case when utilityname = 'FASTLOAD' or utilityname = 'TPTLOAD' or (utilityname = 'FASTEXP' and fastexportnospool = 'Y')
      then phase1totalcputime
      else 0 end as Phase1CPU,
      case when utilityname = 'FASTLOAD' or utilityname = 'TPTLOAD' or (utilityname = 'FASTEXP' and fastexportnospool = 'Y')
      then phase1iokb
      else 0 end as Phase1IO,
      case when utilityname = 'FASTEXP' or utilityname = 'BAR'
      then phase2totalcputime
      else 0 end as Phase2CPU,
      case when utilityname = 'FASTEXP' or utilityname = 'BAR'
      then phase2iokb
      else 0 end as Phase2IO,
      case when utilityname = 'BAR'
      then phase3totalcputime
      else 0 end as Phase3CPU,
      case when utilityname = 'BAR'
      then phase3iokb
      else 0 end as Phase3IO,
      0 as Phase4CPU,
      0 as Phase4IO
    from pdcrinfo.dbqlutilitytbl_hst
    where logdate between DATE-91 and DATE-1
  ) dbu
group by 1, 2) vu4
on
vu1.logdate = vu4.logdate and
vu1.loghr = vu4.loghr ;

Select Site_ID, Count(*) as SiteCount 
from adlste_coa_stg.stg_dat_consumption_vantage_units
group by 1;

Success: 2179 rows affected

In [55]:
Select 'target' as tbl, Count(*) as SiteCount from adlste_coa.coa_dat_consumption_vantage_units    union all 
Select 'stage'  as tbl, Count(*) as SiteCount from adlste_coa_stg.stg_dat_consumption_vantage_units

tbl,SiteCount
target,2225
stage,0


In [50]:
call adlste_coa.sp_dat_consumption_vantage_units('test') ;

Success: 0 rows affected

In [52]:
Select Count(*) as SiteCount from adlste_coa.coa_dat_consumption_vantage_units

SiteCount
2225


In [48]:
insert into adlste_coa_stg.stg_dat_consumption_vantage_units
select * from adlste_coa_stg.stg_dat_consumption_vantage_units_102120201516

Success: 24 rows affected